# Figure out where the navigation instructions lead. What is the Manhattan distance between that location and the ship's starting position?

The navigation instructions (your puzzle input) consists of a sequence of single-character actions paired with integer input values. After staring at them for a few minutes, you work out what they probably mean:

    Action N means to move north by the given value.
    Action S means to move south by the given value.
    Action E means to move east by the given value.
    Action W means to move west by the given value.
    Action L means to turn left the given number of degrees.
    Action R means to turn right the given number of degrees.
    Action F means to move forward by the given value in the direction the ship is currently facing.

For example:

F10
N3
F7
R90
F11

These instructions would be handled as follows:

    F10 would move the ship 10 units east (because the ship starts by facing east) to east 10, north 0.
    N3 would move the ship 3 units north to east 10, north 3.
    F7 would move the ship another 7 units east (because the ship is still facing east) to east 17, north 3.
    R90 would cause the ship to turn right by 90 degrees and face south; it remains at east 17, north 3.
    F11 would move the ship 11 units south to east 17, south 8.

At the end of these instructions, the ship's <b>Manhattan distance</b> (sum of the absolute values of its east/west position and its north/south position) from its starting position is 17 + 8 = 25.

Manhattan distance. <b>Red, blue, yellow</b> are all the <b>shortest</b> Manhattan distance between the points. The green line is the euclidian shortest distance. <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/0/08/Manhattan_distance.svg/1024px-Manhattan_distance.svg.png" alt="Drawing" style="width: 350px;"/>

In [1]:
input_data = []
with open("input.txt") as f:
    input_data = [line.replace("\n", "") for line in f.readlines()]

Declared a dictionary for mapping each character to a function.

In [2]:
def parse_instruction(instruction, state):
    """ Turns the instruction string into the corresponding action. """
    optiuni = {"N": go_north, "S": go_south, "E": go_east, "W": go_west,
               "L": turn_left, "R": turn_right, "F": go_forward}
    caracter = instruction[0]
    number = int(instruction[1:])
    optiuni[caracter](number, state)  # Each function takes the number as parameter.

Declaring all the movement functions. Declaring the initial pozitions.

In [3]:
class State:
    def __init__(self):
        self.x = 0
        self.y = 0
        self.facing = 0  # 0 - East, 1 - North, 2 - West, 3 - South

In [4]:
def go_north(value, state):
    state.y += value

def go_south(value, state):
    state.y -= value

def go_east(value, state):
    state.x += value

def go_west(value, state):
    state.x -= value

def turn_left(value, state):
    value = value / 90
    state.facing += value
    while state.facing > 3:
        state.facing -= 4

def turn_right(value, state):
    value = value / 90
    state.facing -= value
    while state.facing <0:
        state.facing += 4

def go_forward(value, state):
    if state.facing == 0:
        go_east(value, state)
    elif state.facing == 1:
        go_north(value, state)
    elif state.facing == 2:
        go_west(value, state)
    elif state.facing == 3:
        go_south(value, state)

In [5]:
state = State()
for el in input_data:
    parse_instruction(el, state)

state.x, state.y

(678, -1780)

In [6]:
print(abs(state.x) + abs(state.y))

2458


# Figure out where the navigation instructions actually lead. What is the Manhattan distance between that location and the ship's starting position?

Almost all of the actions indicate how to move a waypoint which is relative to the ship's position:

    Action N means to move the waypoint north by the given value.
    Action S means to move the waypoint south by the given value.
    Action E means to move the waypoint east by the given value.
    Action W means to move the waypoint west by the given value.
    Action L means to rotate the waypoint around the ship left (counter-clockwise) the given number of degrees.
    Action R means to rotate the waypoint around the ship right (clockwise) the given number of degrees.
    Action F means to move forward to the waypoint a number of times equal to the given value.

The waypoint starts 10 units east and 1 unit north relative to the ship. The waypoint is relative to the ship; that is, if the ship moves, the waypoint moves with it.

For example, using the same instructions as above:

    F10 moves the ship to the waypoint 10 times (a total of 100 units east and 10 units north), leaving the ship at east 100, north 10. The waypoint stays 10 units east and 1 unit north of the ship.
    N3 moves the waypoint 3 units north to 10 units east and 4 units north of the ship. The ship remains at east 100, north 10.
    F7 moves the ship to the waypoint 7 times (a total of 70 units east and 28 units north), leaving the ship at east 170, north 38. The waypoint stays 10 units east and 4 units north of the ship.
    R90 rotates the waypoint around the ship clockwise 90 degrees, moving it to 4 units east and 10 units south of the ship. The ship remains at east 170, north 38.
    F11 moves the ship to the waypoint 11 times (a total of 44 units east and 110 units south), leaving the ship at east 214, south 72. The waypoint stays 4 units east and 10 units south of the ship.

After these operations, the ship's Manhattan distance from its starting position is 214 + 72 = 286.



Redefining the movement functions. 

In [7]:
def parse_instruction(instruction, state_ship, state_waypoint):
    """ Turns the instruction string into the corresponding action. """
    optiuni = {"N": go_north, "S": go_south, "E": go_east, "W": go_west,
               "L": turn_left, "R": turn_right, "F": go_forward}
    caracter = instruction[0]
    number = int(instruction[1:])
    optiuni[caracter](number, state_ship, state_waypoint)  # Each function takes the number as parameter.

In [8]:
def go_north(value, ship, waypoint):
    waypoint.y += value

def go_south(value, ship, waypoint):
    waypoint.y -= value

def go_east(value, ship, waypoint):
    waypoint.x += value

def go_west(value, ship, waypoint):
    waypoint.x -= value

def turn_left(value, ship, waypoint):
    for i in range(int(value / 90)):
        init_x = waypoint.x
        init_y = waypoint.y
        if ship.facing == 0:
            waypoint.x = -init_y
            waypoint.y = init_x
            ship.facing = 1
        elif ship.facing == 1:
            waypoint.x = -init_y
            waypoint.y = init_x
            ship.facing = 2
        elif ship.facing == 2:
            waypoint.x = -init_y
            waypoint.y = init_x
            ship.facing = 3
        elif ship.facing == 3:
            waypoint.x = -init_y
            waypoint.y = init_x
            ship.facing = 0
        
def turn_right(value, ship, waypoint):
    for i in range(int(value / 90)):
        init_x = waypoint.x
        init_y = waypoint.y
        if ship.facing == 0:
            waypoint.x = init_y
            waypoint.y = -init_x
            ship.facing = 3
        elif ship.facing == 1:
            waypoint.x = init_y
            waypoint.y = -init_x
            ship.facing = 0
        elif ship.facing == 2:
            waypoint.x = init_y
            waypoint.y = -init_x
            ship.facing = 1
        elif ship.facing == 3:
            waypoint.x = init_y
            waypoint.y = -init_x
            ship.facing = 2

def go_forward(value, ship, waypoint):
    ship.x += value * waypoint.x
    ship.y += value * waypoint.y

In [9]:
ship = State()
waypoint = State()
waypoint.x = 10
waypoint.y = 1
for el in input_data:
    parse_instruction(el, ship, waypoint)
    

In [10]:
print(abs(ship.x) + abs(ship.y))

145117
